In [714]:
import os
from pm4py.objects.log.importer.xes import factory as xes_importer
import pandas as pd

log = pd.read_csv(r"C:\Users\Lenovo\Desktop\Magisterka\AlphaMiner\Chapter_1\running-examplee.csv")
data = log.drop(["Event ID", "dd-MM-yyyy:HH.mm", "Resource", "Costs"], axis = 1)
print(log.head())

   Case ID  Event ID  dd-MM-yyyy:HH.mm            Activity Resource  Costs
0        1  35654423  30-12-2010:11.02    register request     Pete     50
1        1  35654424  31-12-2010:10.06  examine thoroughly      Sue    400
2        1  35654425  05-01-2011:15.12        check ticket     Mike    100
3        1  35654426  06-01-2011:11.18              decide     Sara    200
4        1  35654427  07-01-2011:14.24      reject request     Pete    200


1) Zbiór wszystkich tasków

In [715]:
column_values = data[["Activity"]].values.ravel()
tasks =  pd.unique(column_values)
print(tasks)

['register request' 'examine thoroughly' 'check ticket' 'decide'
 'reject request' 'examine casually' 'pay compensation'
 'reinitiate request']


In [716]:
dict = {}
i = 0

for k in tasks:
    dict[k] = i
    i = i + 1
    
print(dict)

{'register request': 0, 'examine thoroughly': 1, 'check ticket': 2, 'decide': 3, 'reject request': 4, 'examine casually': 5, 'pay compensation': 6, 'reinitiate request': 7}


log_tab jest tablica tasks in specific case

In [718]:
import numpy as np
data = data.to_numpy()
elem = data[0][0]
log_tab = []
log2_tab = []

for i in range(len(data)):
    if (data[i][0] != elem or i == (len(data)-1)): 
        if(i == (len(data)-1)):
            log2_tab.append(dict[data[i][1]])
        log2_tab = np.asarray(log2_tab)
        log_tab.append(log2_tab)
        log2_tab = []
        elem = data[i][0]
    log2_tab.append(dict[data[i][1]])
    
print(log_tab)

[array([0, 1, 2, 3, 4]), array([0, 2, 5, 3, 6]), array([0, 5, 2, 3, 7, 1, 2, 3, 6]), array([0, 2, 1, 3, 4]), array([0, 5, 2, 3, 7, 2, 5, 3, 7, 5, 2, 3, 4]), array([0, 5, 2, 3, 6])]


In [719]:
#log_tab = [[0, 1, 6, 7, 9, 10, 8, 11],[0, 2, 3, 4, 5, 6, 9, 7, 8, 10, 11]]
#print(log_tab)

2) Zbiór tasków wejściowych

In [720]:
Tinput = []

for i in range(len(log_tab)):
    if log_tab[i][0] not in Tinput:
        Tinput.append(log_tab[i][0])
        
print(Tinput)

[0]


3) Zbiór tasków wyjściowych

In [721]:
Toutput = []

for i in range(len(log_tab)):
    if log_tab[i][len(log_tab[i])-1] not in Toutput:
        Toutput.append(log_tab[i][len(log_tab[i])-1])
        
print(Toutput)

[4, 6]


4 a) Set of potencial task connections a->b

In [722]:
tuple_tab = []
i = -1

for row in log_tab:
    i = i + 1
    for j in range(len(row)-1):
        tupl = (log_tab[i][j], log_tab[i][j+1])
        if tupl not in tuple_tab:
            tuple_tab.append(tupl)

print(tuple_tab)

[(0, 1), (1, 2), (2, 3), (3, 4), (0, 2), (2, 5), (5, 3), (3, 6), (0, 5), (5, 2), (3, 7), (7, 1), (2, 1), (1, 3), (7, 2), (7, 5)]


Set of potencial task connections b->a

In [723]:
tuple_tab2 = []
i = -1

for row in log_tab:
    i = i + 1
    for j in range(len(row)-1):
        tupl = (log_tab[i][len(row)-1-j], log_tab[i][len(row)-2-j])
        if tupl not in tuple_tab2:
            tuple_tab2.append(tupl)

print(tuple_tab2)

[(4, 3), (3, 2), (2, 1), (1, 0), (6, 3), (3, 5), (5, 2), (2, 0), (1, 7), (7, 3), (2, 5), (5, 0), (3, 1), (1, 2), (5, 7), (2, 7)]


Creating footprint:
    #  = 0
    -> = 1
    <- = 2
    || = 3

In [724]:
footprint = np.zeros((len(tasks), len(tasks)))
#footprint = np.zeros((12, 12))

for i in tuple_tab:
    x = i[0]
    y = i[1]
    footprint[x][y] = 1

for i in tuple_tab2:
    x = i[0]
    y = i[1]
    footprint[x][y] = footprint[x][y] + 2

print(footprint)

[[0. 1. 1. 0. 0. 1. 0. 0.]
 [2. 0. 3. 1. 0. 0. 0. 2.]
 [2. 3. 0. 1. 0. 3. 0. 2.]
 [0. 2. 2. 0. 1. 2. 1. 1.]
 [0. 0. 0. 2. 0. 0. 0. 0.]
 [2. 0. 3. 1. 0. 0. 0. 2.]
 [0. 0. 0. 2. 0. 0. 0. 0.]
 [0. 1. 1. 2. 0. 1. 0. 0.]]


4 b) Finding all triples for which a->(b#c)

In [725]:
tuple_list = []

for i in range(len(footprint)):
    tab_tmp = []
    for j in range(len(footprint)):
        if(footprint[i][j] == 1):
            tab_tmp.append(j)
    if(len(tab_tmp) > 1):
        tab_tmp.append(i)
        tab_tmp = np.asarray(tab_tmp)
        tuple_list.append(tab_tmp)
       
print(tuple_list)     # last element is a number of row, it should be actually e.g. ([1,2,5], 0)    

[array([1, 2, 5, 0]), array([4, 6, 7, 3]), array([1, 2, 5, 7])]


In [726]:
final_triples = []

for i in range(len(tuple_list)):
    for j in range(len(tuple_list[i])-2):
        for k in range(j+1,len(tuple_list[i])-1):
            x = tuple_list[i][j]
            y = tuple_list[i][k]
            final_triples_row = []
            if(footprint[x][y] == 0):
                final_triples_row.append(tuple_list[i][len(tuple_list[i])-1])
                final_triples_row.append(x)
                final_triples_row.append(y)
                if final_triples_row not in final_triples:
                    final_triples.append(final_triples_row)
                    
print(final_triples)     # final_triples is a set of all triples for which a->(b#c), where [a,b,c]

[[0, 1, 5], [3, 4, 6], [3, 4, 7], [3, 6, 7], [7, 1, 5]]


4 c) Finding all triples for which (b#c)->d

In [727]:
tuple_list2 = []

for i in range(len(footprint)):
    tab_tmp2 = []
    for j in range(len(footprint)):
        if(footprint[j][i] == 1):
            tab_tmp2.append(j)
    if(len(tab_tmp2) > 1):
        tab_tmp2.append(i)
        tab_tmp2 = np.asarray(tab_tmp2)
        tuple_list2.append(tab_tmp2)
       
print(tuple_list2)     # last element is a number of row, it should be actually e.g. ([0,7], 1)    

[array([0, 7, 1]), array([0, 7, 2]), array([1, 2, 5, 3]), array([0, 7, 5])]


In [728]:
final_triples2 = []

for i in range(len(tuple_list2)):
    for j in range(len(tuple_list2[i])-2):
        for k in range(j+1,len(tuple_list2[i])-1):
            x = tuple_list2[i][j]
            y = tuple_list2[i][k]
            final_triples_row = []
            if(footprint[x][y] == 0):             
                final_triples_row.append(x)
                final_triples_row.append(y)
                final_triples_row.append(tuple_list2[i][len(tuple_list2[i])-1])
                if final_triples_row not in final_triples2:
                    final_triples2.append(final_triples_row)

print(final_triples2)     # final_triples2 is a set of all triples for which (b#c)->d, where [b,c,d]

[[0, 7, 1], [0, 7, 2], [1, 5, 3], [0, 7, 5]]


5) Eliminating doubled tuples
a) eliminating a->b, a->c if there exists some a->(b#c)

In [729]:
for i in range(len(final_triples)):
    tupl = (final_triples[i][0], final_triples[i][1])
    tupl2 = (final_triples[i][0], final_triples[i][2])
    if tupl in tuple_tab:
        tuple_tab.remove(tupl)
    if tupl2 in tuple_tab:
        tuple_tab.remove(tupl2)  

print(tuple_tab)

[(1, 2), (2, 3), (0, 2), (2, 5), (5, 3), (5, 2), (2, 1), (1, 3), (7, 2)]


b) eliminating b->d, c->d if there exists some (b#c)->d

In [730]:
for i in range(len(final_triples2)):
    tupl = (final_triples2[i][0], final_triples2[i][2])
    tupl2 = (final_triples2[i][1], final_triples2[i][2])
    if tupl in tuple_tab:
        tuple_tab.remove(tupl)
    if tupl2 in tuple_tab:
        tuple_tab.remove(tupl2)  

print(tuple_tab)

[(1, 2), (2, 3), (2, 5), (5, 2), (2, 1)]


* eliminating connections cosed by ||

In [731]:
for i in range(len(footprint)):
    for j in range(len(footprint)):
        if(footprint[i][j] == 3):
            tuple_tab.remove((i, j))


Final set of a->b connections (tuple_tab)

In [732]:
print(tuple_tab)

[(2, 3)]


Final set of a->(b#c) connections (final_triples)

In [733]:
print(final_triples) 

[[0, 1, 5], [3, 4, 6], [3, 4, 7], [3, 6, 7], [7, 1, 5]]


Final set of (b#c)->d connections (final_triples2)

In [735]:
print(final_triples2) 

[[0, 7, 1], [0, 7, 2], [1, 5, 3], [0, 7, 5]]


In [743]:
import bpmn_python.bpmn_diagram_rep as diagram

In [744]:
bpmn_graph = diagram.BPMNDiagramGraph()
bpmn_graph.create_new_diagram_graph(diagram_name="diagram1")

AttributeError: module 'bpmn_python.bpmn_diagram_rep' has no attribute 'BPMNDiagramGraph'